In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import Model

In [ ]:
df = pd.read_csv('./../ejerciciosModelosSupervisados/tasa_conversion/datos/KAG_conversion_data.csv')

In [ ]:
# pre-process datos (si es necesario)

df['age'] = df.age.replace({'30-34':1,'35-39':2,'40-44':3,'45-49':4})
df = df.drop(['ad_id','Approved_Conversion','fb_campaign_id'], axis = 1)
df['Spent'] = df.Spent.astype('float')
df['xyz_campaign_id'] = df.xyz_campaign_id.astype(str)
df = pd.get_dummies(df, drop_first=True)

In [ ]:
yvar = df.Total_Conversion
xvars = df[[x for x in df.columns if x != 'Total_Conversion']]

In [ ]:
# dividir train / test para variables independientes (x)  y dependiente (y)
xtrain, xtest, ytrain, ytest = train_test_split(xvars, yvar, train_size = 0.80, random_state = 2)

In [ ]:
# convertir datos en numpy arrays

xtrain = np.array(xtrain)
xtest= np.array(xtest)
ytrain = np.array(ytrain).squeeze()
ytest = np.array(ytest).squeeze()

In [ ]:
# arquitectura del modelo utilizando variables - FUNCTIONAL API

# input
model_input = Input(shape=(xtrain.shape[1],))
# primera capa
layer_1 = Dense(256, kernel_regularizer='l2', activation='relu')(model_input)
#segunda capa
layer_2 = Dense(256, kernel_regularizer='l2', activation='relu')(layer_1)
# predicción
model_output = Dense(1)(layer_2)
# entrada y salida
model = Model(inputs=model_input, 
              outputs=model_output)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0002)

model.compile(optimizer=optimizer, loss='mae')

In [ ]:
history = model.fit(
    xtrain,
    ytrain,
    epochs=200,
    callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss',verbose=2, min_delta=0.001, patience=10),
                 tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=5)],
    validation_data=(xtest, ytest)
)

In [ ]:
fig, ax = plt.subplots(figsize=(12,4))
ax.plot(range(len(history.history['loss'])), history.history['loss'], label='train loss')
ax.plot(range(len(history.history['val_loss'])), history.history['val_loss'], label='train loss')
ax.legend()
plt.ylabel("loss")
plt.title('Loss at Training')
plt.show()